In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [4]:
dat = pd.read_csv("pacifier_cleaned.csv", header=0)

In [102]:
from sklearn.feature_extraction import text
my_stopwords = text.ENGLISH_STOP_WORDS.union(['baby','pacifier','pacifiers','product','did','does','do','just',
                                              'buy','bought'])
text.CountVectorizer(stop_words= my_stopwords)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}),
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [126]:
y_dict = {1:0, 2:0, 3:2, 4:1, 5:1}
dat['emotion'] = dat['star_rating'].map(y_dict)
dat = dat.fillna("blank")

pos = dat[dat['emotion'] == 1]
pos = pos.reset_index(drop=True)

neg = dat[dat['emotion'] == 0]
neg = neg.reset_index(drop=True)

In [127]:
pos = pos['review_body']
neg = neg['review_body']

In [105]:
# pos[pos.isnull().values==True]

In [106]:
# to record the attributes and words
attr = {}
# the attributes we need
select_attr = ['JJ', 'JJR', 'JJS', 'NN', 'NNP', 'NNS', 'VB', 'VBZ', 'VBD', 'VBN', 'RBR']

# Word Segmentation and POS Tagging -- select words
for i in range(len(pos)):
    # cut sentences
    sentence = change_form(word_tokenize(pos[i]))
    # tag the attributes; select words by some conditions
    words_attr = [(x[0],x[1]) for x in pos_tag(sentence) if x[1] in select_attr and x[0].lower() not in my_stopwords and check_English(x[0].lower())]
    # add words; group by attributes
    for x in words_attr:
        if x[1] in attr.keys():
            if x[0] not in attr[x[1]]:
                attr[x[1]].append(x[0].lower())
        else:
            attr[x[1]] = [x[0].lower()]

# extract the words
words = list(attr.values())
# combine words in one list
words_whole = []
for i in range(len(words)):
    words_whole.extend(words[i])

# select the words in words_whole list
def handle(x):
    temp = [y for y in x if y in words_whole]
    return temp

# cut each sentence in review_body; put the result(words) in a list 
cut_sentences = [handle(word_tokenize(x)) for x in pos]

res = []
for li in cut_sentences:
    s = " ".join(li)
    res.append(s)

pos_content = pd.Series(res)

In [107]:
# to record the attributes and words
attr = {}
# the attributes we need
select_attr = ['JJ', 'JJR', 'JJS', 'NN', 'NNP', 'NNS', 'VB', 'VBZ', 'VBD', 'VBN', 'RBR']

# Word Segmentation and POS Tagging -- select words
for i in range(len(neg)):
    # cut sentences
    sentence = change_form(word_tokenize(neg[i]))
    # tag the attributes; select words by some conditions
    words_attr = [(x[0],x[1]) for x in pos_tag(sentence) if x[1] in select_attr and x[0].lower() not in my_stopwords and check_English(x[0].lower())]
    # add words; group by attributes
    for x in words_attr:
        if x[1] in attr.keys():
            if x[0] not in attr[x[1]]:
                attr[x[1]].append(x[0].lower())
        else:
            attr[x[1]] = [x[0].lower()]

# extract the words
words = list(attr.values())
# combine words in one list
words_whole = []
for i in range(len(words)):
    words_whole.extend(words[i])

# select the words in words_whole list
def handle(x):
    temp = [y for y in x if y in words_whole]
    return temp

# cut each sentence in review_body; put the result(words) in a list 
cut_sentences = [handle(word_tokenize(x)) for x in neg]

res = []
for li in cut_sentences:
    s = " ".join(li)
    res.append(s)

neg_content = pd.Series(res)

In [113]:
from gensim import corpora,models

data_positive = pos_content.apply(lambda x:x.split(' '))

pos_dict = corpora.Dictionary(data_positive)
pos_corpus = [pos_dict.doc2bow(i) for i in data_positive]
pos_lda = models.LdaModel(corpus=pos_corpus,num_topics=8,id2word=pos_dict)

print(pos_lda.print_topics(8))

[(0, '0.030*"loves" + 0.029*"son" + 0.019*"mouth" + 0.015*"little" + 0.014*"use" + 0.014*"animal" + 0.013*"hold" + 0.013*"daughter" + 0.012*"ones" + 0.010*"great"'), (1, '0.033*"loves" + 0.028*"cute" + 0.025*"daughter" + 0.022*"mouth" + 0.022*"great" + 0.020*"paci" + 0.017*"son" + 0.016*"little" + 0.014*"hold" + 0.014*"old"'), (2, '0.018*"price" + 0.018*"great" + 0.014*"perfect" + 0.013*"love" + 0.013*"like" + 0.012*"hospital" + 0.012*"shipping" + 0.012*"ones" + 0.010*"fast" + 0.008*"rubber"'), (3, '0.014*"paci" + 0.013*"got" + 0.013*"little" + 0.012*"really" + 0.010*"use" + 0.009*"easy" + 0.008*"gift" + 0.008*"big" + 0.008*"loved" + 0.008*"shower"'), (4, '0.018*"binky" + 0.017*"months" + 0.016*"use" + 0.014*"son" + 0.013*"like" + 0.012*"child" + 0.012*"mouth" + 0.011*"soothie" + 0.010*"time" + 0.010*"daughter"'), (5, '0.043*"love" + 0.036*"" + 0.017*"son" + 0.016*"like" + 0.015*"mouth" + 0.014*"loved" + 0.012*"face" + 0.010*"shape" + 0.009*"avent" + 0.009*"loves"'), (6, '0.037*"like" 

In [116]:
from gensim import corpora,models

data_negative = neg_content.apply(lambda x:x.split(' '))

neg_dict = corpora.Dictionary(data_negative)
neg_corpus = [neg_dict.doc2bow(i) for i in data_negative]
#放进gensim.models中的lda模型进行训练
neg_lda = models.LdaModel(corpus=neg_corpus,num_topics=8,id2word=neg_dict)

print(neg_lda.print_topics(8))

[(0, '0.025*"like" + 0.011*"paci" + 0.010*"really" + 0.010*"son" + 0.009*"got" + 0.008*"color" + 0.008*"hard" + 0.008*"old" + 0.008*"plastic" + 0.007*"good"'), (1, '0.012*"mouth" + 0.011*"use" + 0.009*"like" + 0.009*"nipple" + 0.008*"little" + 0.008*"babies" + 0.007*"daughter" + 0.006*"better" + 0.006*"big" + 0.006*"son"'), (2, '0.011*"like" + 0.011*"little" + 0.010*"pink" + 0.010*"use" + 0.009*"nipple" + 0.009*"really" + 0.008*"mouth" + 0.006*"time" + 0.006*"wash" + 0.006*"ones"'), (3, '0.011*"son" + 0.010*"month" + 0.010*"big" + 0.010*"different" + 0.009*"used" + 0.009*"cute" + 0.009*"old" + 0.008*"mouth" + 0.008*"right" + 0.007*"babies"'), (4, '0.016*"use" + 0.010*"mouth" + 0.010*"bag" + 0.010*"son" + 0.009*"got" + 0.009*"like" + 0.008*"great" + 0.006*"really" + 0.006*"colors" + 0.006*"diaper"'), (5, '0.014*"months" + 0.013*"son" + 0.012*"use" + 0.011*"month" + 0.011*"old" + 0.011*"mouth" + 0.010*"time" + 0.010*"child" + 0.009*"good" + 0.008*"got"'), (6, '0.023*"like" + 0.019*"mouth